In [1]:
import pandas as pd
import re 
import numpy
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import StopWordsRemover
from pyspark.sql.functions import * 
from pyspark.sql.functions import length
from pyspark.sql.functions import udf
from pyspark.sql import functions as f
from pyspark.ml.feature import CountVectorizer
from pyspark.sql.functions import col, when
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml.feature import StringIndexer


In [2]:
spark=SparkSession.builder.appName('mmarulandc').getOrCreate()
csv = '/user/mmarulandc/datasets/articles2.csv'
df1 = spark.read.csv(csv).toPandas()
df1.head(2)

,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9
0,None,id,title,publication,author,date,year,month,url,content
1,53293,73471,Patriots Day Is Best When It Digs Past the Her...,Atlantic,David Sims,2017-01-11,2017.0,1.0,None,"Patriots Day, Peter Berg’s new thriller that r..."


In [3]:
#Creando un dataframe que contenga los contedidos de las publicaciones hechas y limpiando el contenido de caracteres especiales.
df1['_cleaned'] = [re.sub(r"[^a-zA-Z0-9]+",' ', str(x)) for x in df1['_c9']]
df1.head()


,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,_cleaned
0,None,id,title,publication,author,date,year,month,url,content,content
1,53293,73471,Patriots Day Is Best When It Digs Past the Her...,Atlantic,David Sims,2017-01-11,2017.0,1.0,None,"Patriots Day, Peter Berg’s new thriller that r...",Patriots Day Peter Berg s new thriller that re...
2,53294,73472,A Break in the Search for the Origin of Comple...,Atlantic,Ed Yong,2017-01-11,2017.0,1.0,None,"In Norse mythology, humans and our world were ...",In Norse mythology humans and our world were c...
3,53295,73474,Obama’s Ingenious Mention of Atticus Finch,Atlantic,Spencer Kornhaber,2017-01-11,2017.0,1.0,None,“If our democracy is to work in this increasin...,If our democracy is to work in this increasin...
4,53296,73475,"Donald Trump Meets, and Assails, the Press",Atlantic,David A. Graham,2017-01-11,2017.0,1.0,None,Updated on January 11 at 5:05 p. m. In his fir...,Updated on January 11 at 5 05 p m In his first...


In [4]:
#Para verificar que el procesamiento ocurrio correctamente, haremos un word cloud usando la libreria wordcloud para obtener una representacion visual de las palabras más comunes, esta es la clave para enteder la data y que vamos en el buen camino para el entrenamiento de modelos


In [5]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
count_vectorizer = CountVectorizer(stop_words='english')
# Fit and transform the processed titles
count_data = count_vectorizer.fit_transform(df1['_cleaned'])

In [6]:
import warnings
warnings.simplefilter("ignore", DeprecationWarning)
# Load the LDA model from sk-learn
from sklearn.decomposition import LatentDirichletAllocation as LDA
 
# Helper function
def print_topics(model, count_vectorizer, n_top_words):
    words = count_vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([words[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
        
# Tweak the two parameters below
number_topics = 5
number_words = 10
# Create and fit the LDA model
lda = LDA()
lda.fit(count_data)
# Print the topics found by the LDA model
print("Topics found via LDA:")
print_topics(lda, count_vectorizer, number_words)

Topics found via LDA:

Topic #0:
said police people city told killed man officers attack death

Topic #1:
people women school students like work percent said just don

Topic #2:
said just like time game going season says don team

Topic #3:
trump said president obama house state administration government white states

Topic #4:
said new million york city company court 000 told post

Topic #5:
says people health said eu years study research university medical

Topic #6:
trump clinton donald campaign said president republican party people hillary

Topic #7:
trump new president company business said million year percent china

Topic #8:
like film new world music life just song black movie

Topic #9:
like time new just people world years facebook way apple
